In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from pprint import pprint
%matplotlib inline
data_path = "covid10_08"
data = pd.read_csv(data_path)
#validation = pd.read_csv("covid10_01")
Countries = ["Russia","Greece","India","United_States_of_America","Turkey"]

In [5]:
def MAPE(val,y_pred):
  score = 0
  for i in range(0,7):
    if val[i] != 0:
      score += abs((val[i]-y_pred[i])/val[i])
  return ((1/7)*score)
def Average(lst): 
    new_lst = []
    for i in range(0,len(lst)-1):
      if lst[i]<2:
        new_lst.append(lst[i])
    return sum(new_lst) / len(new_lst) 
def average(lst): 
    return sum(lst) / len(lst) 

In [6]:
data = pd.read_csv(data_path)
import warnings
warnings.filterwarnings('ignore')
best_days_list = []
for i in range(0,len(Countries)):
  MAPE_score = []
  for j in range(15,150):
    pred_days = j
    if (data[data['countriesAndTerritories']== Countries[i]]['cases'].count() > 150):
      #pred_days = 53
      test_days = 7
      train_days = pred_days - test_days
      #final = validation[validation['countriesAndTerritories']=='Uruguay']['cases'][ : 5][::-1]
      reverse_data = data[data['countriesAndTerritories']==Countries[i]].iloc[::-1]
      train_data = reverse_data[reverse_data['countriesAndTerritories']==Countries[i]][-1*(pred_days):]
      df = train_data[train_data['countriesAndTerritories']==Countries[i]]['cases'][ : -1*(test_days)]
      #df = train_data[train_data['countriesAndTerritories']==Countries[i]]['cases']
      val = train_data[train_data['countriesAndTerritories']==Countries[i]]['cases'][-1*(test_days):]
      df = df.to_numpy()
      val = val.to_numpy()
      y = df.reshape(-1,1)
      x = np.reshape(list(range(1,pred_days-test_days+1)), (-1,1))
      sc_X = StandardScaler()
      sc_y = StandardScaler()
      X = sc_X.fit_transform(x)
      Y = sc_y.fit_transform(y)
      regressor = SVR(kernel='rbf')
      regressor.fit(X,Y)
      X_ans = np.reshape(list(range(pred_days-6,pred_days+1)), (-1,1))
      y_pred = sc_y.inverse_transform((regressor.predict (sc_X.transform(X_ans))))
      for k in range(0,7):
        y_pred[k] = round(y_pred[k])
      #answers[Countries[i]] = y_pred
      #print(Countries[i],y_pred[:7])
      MAPE_score.append(MAPE(val,y_pred))
    else:
      MAPE_score = [10,10,10,10,10] 
  idx = np.argsort(MAPE_score)[:5]
  print(Countries[i],":",idx[:5],"Mape score:", MAPE_score[idx[0]],MAPE_score[idx[1]],MAPE_score[idx[2]])
  best_days_list.append(idx[0])

Russia : [38 31 37 39 30] Mape score: 0.20069901007637808 0.20137014474592022 0.20248964325224764
Greece : [ 1  0  4  3 33] Mape score: 0.22168301947005506 0.22168728206721336 0.22237506439553756
India : [82 79 78 81 77] Mape score: 0.05360690824976466 0.05375484211405808 0.053866296352987106
United_States_of_America : [ 2  3 54 55 59] Mape score: 0.1119786527781404 0.11422764703171136 0.11453259458970652
Turkey : [12 10 15 14 11] Mape score: 0.023260941622140638 0.02406197597094329 0.02478529726168633


In [11]:
import warnings
data = pd.read_csv(data_path)
warnings.filterwarnings('ignore')
Rows = ["10/9","10/10","10/11","10/12","10/13","10/14","10/15"]
answers = pd.DataFrame(index=Rows,columns=Countries)
MAPE_score = []
for i in range(0,len(Countries)):
  pred_days = best_days_list[i]+10
  test_days = 0
  train_days = pred_days - test_days
  if (data[data['countriesAndTerritories']== Countries[i]]['cases'].count() < 150):
    pred_days = data[data['countriesAndTerritories']==Countries[i]]['cases'].count()
  #final = validation[validation['countriesAndTerritories']=='Uruguay']['cases'][ : 5][::-1]
  reverse_data = data[data['countriesAndTerritories']==Countries[i]].iloc[::-1]
  train_data = reverse_data[reverse_data['countriesAndTerritories']==Countries[i]][-1*(pred_days):]
  #df = train_data[train_data['countriesAndTerritories']==Countries[i]]['cases'][ : -1*(test_days)]
  df = train_data[train_data['countriesAndTerritories']==Countries[i]]['cases']
  val = train_data[train_data['countriesAndTerritories']==Countries[i]]['cases'][-1*(test_days):]
  df = df.to_numpy()
  val = val.to_numpy()
  y = df.reshape(-1,1)
  x = np.reshape(list(range(1,pred_days-test_days+1)), (-1,1))
  sc_X = StandardScaler()
  sc_y = StandardScaler()
  X = sc_X.fit_transform(x)
  Y = sc_y.fit_transform(y)
  regressor = SVR(kernel='rbf')
  regressor.fit(X,Y)
  X_ans = np.reshape(list(range(pred_days-6,pred_days+1)), (-1,1))
  y_pred = sc_y.inverse_transform((regressor.predict (sc_X.transform(X_ans))))
  for j in range(0,7):
    y_pred[j] = round(y_pred[j])
  answers[Countries[i]] = y_pred
  #print(Countries[i],y_pred[:7])
  #MAPE_score.append(MAPE(val,y_pred))
#avg_score = Average(MAPE_score)


In [12]:
#print(avg_score)
for i in range(len(Countries)):
  for j in range(7):
    if answers[Countries[i]][j]<0:
      print(Countries[i])
      answers[Countries[i]][j] = 0
answers.to_csv(r"106022113_HW1.csv")
answers##Final Answer

,Russia,Greece,India,United_States_of_America,Turkey
10/9,9187.0,366.0,79475.0,44609.0,1415.0
10/10,9557.0,337.0,78346.0,45450.0,1436.0
10/11,9906.0,298.0,77232.0,44873.0,1465.0
10/12,10225.0,283.0,76140.0,43678.0,1496.0
10/13,10504.0,308.0,75076.0,43072.0,1526.0
10/14,10736.0,356.0,74046.0,43632.0,1550.0
10/15,10915.0,395.0,73053.0,44895.0,1568.0
